# PCASL processing

In [12]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
from nibabel import load, Nifti1Image

from nipype.interfaces.slicer.registration import brainsresample
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.freesurfer.preprocess import MRIConvert, ApplyVolTransform, MNIBiasCorrection
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.fsl.maths import ApplyMask
from nipype.interfaces.fsl.preprocess import FLIRT
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.spm.preprocess import Smooth
from nipype.interfaces.ants.registration import Registration
from nipype.interfaces.ants.resampling import ApplyTransforms

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#other study-specific variables
project_home = '/Users/catcamacho/Box/SNAP/BABIES'
raw_dir = project_home + '/raw'
subjects_list = open(project_home + '/misc/subjects_asl.txt').read().splitlines()
#subjects_list = ['105', '106','102', '100','099','091','1005','1010', '1020','1033']
output_dir = project_home + '/proc/asl_preproc'
wkflow_dir = project_home + '/workflows'
template = project_home + '/templates/T2w_BABIES_template_2mm.nii.gz'
mask = project_home + '/templates/T2w_BABIES_template_2mm_mask.nii.gz'
#T1_file = project_home + '/templates/qT1_template_2mm.nii.gz' 
#T1_vol = load(T1_file)
#T1_tissue = T1_vol.get_data()
T1_tissue = 1.4 #estimated T1 of grey matter in seconds

#Population specific variables for ASL
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header at position 0043,107f (corresponds to #coils?)
postlabel_delay = 1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
TR = 4.844 #repetition time

In [13]:
## File handling nodes

# Select subjects
infosource = Node(IdentityInterface(fields=['subjid', 'volume']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]


# SelectFiles
templates = {'pw_volume': raw_dir + '/{subjid}-BABIES/pw.nii.gz',
             'anat_volume': raw_dir + '/{subjid}-BABIES/skullstripped_anat.nii.gz',
             'pd_volume': raw_dir + '/{subjid}-BABIES/pd.nii.gz'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')


# Datasink
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
# DataSink output substitutions (for ease of folder naming)
substitutions = [('_subjid_', ''),
                ('volume_',''),
                ('_reoriented',''),
                ('_warped','')]
datasink.inputs.substitutions = substitutions

In [14]:
## File Processing nodes

# convert files to nifti
mri_convert = Node(MRIConvert(out_type='niigz',
                              conform_size=2,
                              crop_size=(128, 128, 128), 
                             ),
                   name='mri_convert')

# reorient data for consistency
reorient_anat = Node(Reorient2Std(),
                     name='reorient_anat')

# reorient data for consistency
reorient_pd = Node(Reorient2Std(),
                   name='reorient_pd')

# reorient data for consistency
reorient_pw = Node(Reorient2Std(),
                   name='reorient_pw')

# N3 bias correction using MINC tools
nu_correct_pd = Node(MNIBiasCorrection(iterations=4, 
                                       no_rescale=True, 
                                       out_file='pd_nu.nii.gz'), 
                     name='nu_correct_pd')

nu_correct_pw = Node(MNIBiasCorrection(iterations=4, 
                                       no_rescale=True, 
                                       out_file='pw_nu.nii.gz'), 
                     name='nu_correct_pw')

# register PW to anat
coregT2 = Node(Registration(args='--float',
                            collapse_output_transforms=True,
                            initial_moving_transform_com=True,
                            num_threads=1,
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            sigma_units=['vox']*2,
                            transforms=['Rigid', 'Affine'],
                            terminal_output='file',
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            convergence_threshold=[1e-06],
                            convergence_window_size=[10],
                            metric=['Mattes', 'Mattes'],
                            metric_weight=[1.0]*2,
                            number_of_iterations=[[100, 75, 50],
                                                  [100, 75, 50]],
                            radius_or_number_of_bins=[32, 32],
                            sampling_percentage=[0.25, 0.25],
                            sampling_strategy=['Regular',
                                               'Regular'],
                            shrink_factors=[[4, 2, 1]]*2,
                            smoothing_sigmas=[[2, 1, 0]]*2,
                            transform_parameters=[(0.1,),
                                                  (0.1,)],
                            use_histogram_matching=False,
                            write_composite_transform=True),
               name='coregT2')


applyxform = Node(ApplyTransforms(), name = 'applyxform')

In [15]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

In [16]:
# Create a flow for preprocessing anat + asl volumes 
preprocflow = Workflow(name='preprocflow')

# Connect all components of the preprocessing workflow
preprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, mri_convert, [('anat_volume', 'in_file')]),
                     (mri_convert, reorient_anat, [('out_file', 'in_file')]),    
                     (selectfiles, reorient_pw, [('pw_volume', 'in_file')]), 
                     (selectfiles, reorient_pd, [('pd_volume', 'in_file')]),
                     (reorient_anat, coregT2, [('out_file', 'fixed_image')]),
                     (reorient_pw, nu_correct_pw, [('out_file','in_file')]),
                     (nu_correct_pw, coregT2, [('out_file', 'moving_image')]),
                     (coregT2, applyxform, [('composite_transform','transforms')]),
                     (reorient_anat, applyxform, [('out_file','reference_image')]),
                     (reorient_pd, nu_correct_pd, [('out_file','in_file')]),
                     (nu_correct_pd, applyxform, [('out_file','input_image')]),
                     (coregT2, make_coreg_img, [('warped_image','epi')]),
                     (reorient_anat, make_coreg_img, [('out_file','anat')]),
                     
                     (make_coreg_img, datasink, [('coreg_file','coreg_check')]),
                     (applyxform, datasink, [('output_image','warped_pd')]),
                     (coregT2, datasink, [('warped_image','warped_pw')]),
                     (reorient_anat, datasink, [('out_file', 'reorient_anat')])
                    ])
preprocflow.base_dir = wkflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 2})


180619-21:27:29,30 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/graph.png (graph2use=flat, simple_form=True).
180619-21:27:29,65 workflow INFO:
	 Workflow preprocflow settings: ['check', 'execution', 'logging', 'monitoring']
180619-21:27:29,427 workflow INFO:
	 Running in parallel.
180619-21:27:29,451 workflow INFO:
	 [MultiProc] Running 0 tasks, and 39 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
180619-21:27:29,537 workflow INFO:
	 [Node] Setting-up "preprocflow.selectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/selectfiles".
180619-21:27:29,537 workflow INFO:
	 [Node] Setting-up "preprocflow.selectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_027/selectfiles".
180619-21:27:29,551 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
180619-21:27:29,551 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


180619-21:29:28,898 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/_0x098cb29d5ba7fd74d2f84eb15a3b0031_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/_node.pklz
180619-21:29:28,900 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/_report
180619-21:29:28,903 workflow DEBUG:
	 Removing files: 
180619-21:29:28,910 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/result_make_coreg_img.pklz
180619-21:29:28,913 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_062/make_coreg_img/_r

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


180619-21:31:21,674 workflow INFO:
	 [MultiProc] Running 2 tasks, and 36 jobs ready. Free memory (GB): 14.00/14.40, Free processors: 0/2.                     
                     Currently running:
                       * preprocflow.make_coreg_img
                       * preprocflow.coregT2


...

180619-22:09:26,522 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/_0x72cd49afd6f2117ca921da40a92d1c6b_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/_node.pklz
180619-22:09:26,524 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/_report
180619-22:09:26,526 workflow DEBUG:
	 Removing files: 
180619-22:09:26,533 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/result_make_coreg_img.pklz
180619-22:09:26,535 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_035/make_coreg_img/_r

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


180619-22:10:10,171 workflow INFO:
	 [Node] Finished "preprocflow.applyxform".
180619-22:10:10,544 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/_0xb6d3c947e028760a45ea831e2eaaeb3f_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/_node.pklz
180619-22:10:10,547 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/_report
180619-22:10:10,548 workflow DEBUG:
	 Removing files: 
180619-22:10:10,554 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_033x/make_coreg_img/result_make_coreg_img.pklz
180619-22:10:10,556 workflow DEBUG:
	 [Node] Writing post-exec report to "

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


180619-22:12:56,531 workflow INFO:
	 [Node] Finished "preprocflow.applyxform".
180619-22:12:56,934 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/_0x92257c76b3877367a96fe837b123c8b5_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/_node.pklz
180619-22:12:56,936 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/_report
180619-22:12:56,938 workflow DEBUG:
	 Removing files: 
180619-22:12:56,943 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocflow/_subjid_1024/make_coreg_img/result_make_coreg_img.pklz
180619-22:12:56,945 workflow DEBUG:
	 [Node] Writing post-exec report to "

In [17]:
## File handling nodes for CBF proc

# Select subjects
cbfinfosource = Node(IdentityInterface(fields=['subjid']),
                  name='cbfinfosource')
cbfinfosource.iterables = [('subjid', subjects_list)]


# SelectFiles
templates = {'pw_volume': output_dir + '/warped_pw/{subjid}/transform_Warped.nii.gz',
            'pd_volume': output_dir + '/warped_pd/{subjid}/pd_nu_trans.nii.gz',
            'anat_volume': output_dir + '/reorient_anat/{subjid}/skullstripped_anat_out.nii.gz'}
cbfselectfiles = Node(SelectFiles(templates), name='cbfselectfiles')

In [18]:
## Custom functions

#quantify CBF from PW volume (Alsop MRIM 2015 method)
def quantify_cbf_alsop(pw_volume,pd_volume,sat_time,postlabel_delay,T1_blood,labeling_time,efficiency,partition_coeff,TR,T1_tissue,scaling_factor,nex_asl):
    import os
    import nibabel as nib
    from numpy import exp
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    # set variables
    pw_nifti1 = nib.load(pw_volume)
    pw_data = pw_nifti1.get_data()
    pw_data = pw_data.astype(float)
    pd_nifti1 = nib.load(pd_volume)
    pd_data = pd_nifti1.get_data()
    pd_data = pd_data.astype(float)
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    pd_factor = 1/(1-exp((-1*TR)/T1_tissue))
    
    cbf_numerator = conversion*partition_coeff*pw_data*exp(postlabel_delay/T1_blood)
    cbf_denominator = sat_time*efficiency*T1_blood*scaling_factor*nex_asl*pd_data*pd_factor*(1-exp((-1*labeling_time)/T1_blood))
    cbf_data = cbf_numerator/cbf_denominator
    
    cbf_volume = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_volume, 'alsop_cbf.nii')
    cbf_path = os.path.abspath('alsop_cbf.nii')
    return cbf_path

quant_cbf_alsop = Node(name='quant_cbf_alsop',
                interface=Function(input_names=['pw_volume','pd_volume',
                                                'sat_time','postlabel_delay',
                                                'T1_blood','labeling_time',
                                                'efficiency','partition_coeff',
                                                'TR','T1_tissue','scaling_factor',
                                                'nex_asl'],
                                  output_names=['cbf_volume'],
                                  function=quantify_cbf_alsop))
quant_cbf_alsop.inputs.sat_time=sat_time
quant_cbf_alsop.inputs.postlabel_delay=postlabel_delay
quant_cbf_alsop.inputs.T1_blood=T1_blood
quant_cbf_alsop.inputs.labeling_time=labeling_time
quant_cbf_alsop.inputs.efficiency=efficiency
quant_cbf_alsop.inputs.partition_coeff=partition_coeff
quant_cbf_alsop.inputs.TR=TR
quant_cbf_alsop.inputs.T1_tissue=T1_tissue
quant_cbf_alsop.inputs.scaling_factor=scaling_factor
quant_cbf_alsop.inputs.nex_asl=nex_asl

In [19]:
## Normalizing data for first and second level analysis
smooth = Node(Smooth(fwhm=[4,4,4], 
                     implicit_masking=True), 
              name='smooth')

# register PW to anat
reg2temp = Node(Registration(args='--float',
                             collapse_output_transforms=True,
                             initial_moving_transform_com=True,
                             num_threads=1,
                             output_inverse_warped_image=True,
                             output_warped_image=True,
                             sigma_units=['vox']*2,
                             transforms=['Rigid', 'Affine'],
                             terminal_output='file',
                             winsorize_lower_quantile=0.005,
                             winsorize_upper_quantile=0.995,
                             convergence_threshold=[1e-06],
                             convergence_window_size=[10],
                             metric=['Mattes', 'Mattes', 'CC'],
                             metric_weight=[1.0]*3,
                             number_of_iterations=[[100, 75, 50],
                                                   [100, 75, 50]],
                             radius_or_number_of_bins=[32, 32],
                             sampling_percentage=[0.25, 0.25],
                             sampling_strategy=['Regular',
                                                'Regular'],
                             shrink_factors=[[4, 2, 1]]*2,
                             smoothing_sigmas=[[2, 1, 0]]*2,
                             transform_parameters=[(0.1,),
                                                   (0.1,)],
                             use_histogram_matching=False,
                             write_composite_transform=True, 
                             fixed_image=template),
                name='reg2temp')

applyxform_pw = Node(ApplyTransforms(reference_image=template), 
                     name = 'applyxform_pw')
applyxform_pd = Node(ApplyTransforms(reference_image=template), 
                     name = 'applyxform_pd')

apply_mask = Node(ApplyMask(mask_file=mask),name='apply_mask')

# Check template registration
check_template_coreg = Node(name='check_template_coreg',
                            interface=Function(input_names=['epi','anat'],
                                               output_names=['coreg_file'],
                                               function=create_coreg_plot))
check_template_coreg.inputs.anat=template

In [20]:
# create a flow for quantifying CBF and warping to MNI space.
cbfprocflow = Workflow(name='cbfprocflow')

# connect the nodes
cbfprocflow.connect([(cbfinfosource, cbfselectfiles, [('subjid', 'subjid')]),
                     (cbfselectfiles, reg2temp, [('anat_volume', 'moving_image')]),
                     (cbfselectfiles, applyxform_pw, [('pw_volume','input_image')]),
                     (cbfselectfiles, applyxform_pd, [('pd_volume','input_image')]),
                     (reg2temp, applyxform_pw, [('composite_transform','transforms')]),
                     (reg2temp, applyxform_pd, [('composite_transform','transforms')]),
                     (applyxform_pw, quant_cbf_alsop, [('output_image', 'pw_volume')]),
                     (applyxform_pd, quant_cbf_alsop, [('output_image', 'pd_volume')]),
                     (quant_cbf_alsop, apply_mask, [('cbf_volume','in_file')]),
                     (apply_mask, check_template_coreg, [('out_file','epi')]),
                     
                     (reg2temp, datasink, [('warped_image','processed_t2')]),
                     (check_template_coreg, datasink, [('coreg_file','template_coreg')]),
                     (apply_mask, datasink, [('out_file', 'cbf_volume')])
                    ]),
cbfprocflow.base_dir = wkflow_dir
cbfprocflow.write_graph(graph2use='flat')
cbfprocflow.run('MultiProc', plugin_args={'n_procs': 2})


180619-22:13:02,543 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/graph.png (graph2use=flat, simple_form=True).
180619-22:13:02,576 workflow INFO:
	 Workflow cbfprocflow settings: ['check', 'execution', 'logging', 'monitoring']
180619-22:13:02,881 workflow INFO:
	 Running in parallel.
180619-22:13:02,897 workflow INFO:
	 [MultiProc] Running 0 tasks, and 39 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
180619-22:13:03,0 workflow INFO:
	 [Node] Setting-up "cbfprocflow.cbfselectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1010/cbfselectfiles".
180619-22:13:02,999 workflow INFO:
	 [Node] Setting-up "cbfprocflow.cbfselectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_091/cbfselectfiles".
180619-22:13:03,16 workflow INFO:
	 [Node] Running "cbfselectfiles" ("nipype.interfaces.io.SelectFiles")
180619-22:13:03,16 workflow INFO:
	 [Node] Running "cbfselectfiles" ("nip

<string>:21: RuntimeWarning: divide by zero encountered in divide
<string>:21: RuntimeWarning: invalid value encountered in divide


180619-22:15:47,708 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/alsop_cbf.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/alsop_cbf.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/_0x3d286274c0153087b40b77ce815b0d8e_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/_node.pklz
180619-22:15:47,710 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/_report
180619-22:15:47,712 workflow DEBUG:
	 Removing files: 
180619-22:15:47,719 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/quant_cbf_alsop/result_quant_cbf_alsop.pklz
180619-22:15:47,721 workflow DEBUG:
	 [Node] Writing post-e

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/usr/local/lib/python2.7/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/usr/local/lib/python2.7/site-packages/nilearn/plotting/img_plotting.py:142: UserWarni

180619-22:15:52,361 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/_0x1fa6ed8cef7e18e75dd30ffdb3cca120_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/_node.pklz
180619-22:15:52,363 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/_report
180619-22:15:52,365 workflow DEBUG:
	 Removing files: 
180619-22:15:52,373 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_087/check_template_coreg/result_check_template_coreg.pklz
180619-22:15:52,375 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/workflows

...

180619-22:52:49,911 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/_0x2815499b9d97dd64dd362c954a0a25c3_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/_node.pklz
180619-22:52:49,913 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/_report
180619-22:52:49,915 workflow DEBUG:
	 Removing files: 
180619-22:52:49,921 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_1036/check_template_coreg/result_check_template_coreg.pklz
180619-22:52:49,923 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/wor

<string>:21: RuntimeWarning: divide by zero encountered in divide
<string>:21: RuntimeWarning: invalid value encountered in divide


180619-22:54:17,465 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/alsop_cbf.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/alsop_cbf.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/_0xadf9e2516d42509aa805be9e414e6f77_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/_node.pklz
180619-22:54:17,468 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/_report
180619-22:54:17,470 workflow DEBUG:
	 Removing files: 
180619-22:54:17,476 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/quant_cbf_alsop/result_quant_cbf_alsop.pklz
180619-22:54:17,478 workflow DEBUG:
	 [Node] Writing post-e

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/usr/local/lib/python2.7/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/usr/local/lib/python2.7/site-packages/nilearn/plotting/img_plotting.py:142: UserWarni

180619-22:54:22,130 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/_0xc74a43cee70154cd650e3491d8335dd7_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/_node.pklz
180619-22:54:22,132 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/_report
180619-22:54:22,134 workflow DEBUG:
	 Removing files: 
180619-22:54:22,140 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/cbfprocflow/_subjid_110/check_template_coreg/result_check_template_coreg.pklz
180619-22:54:22,142 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/workflows